In [17]:
import pymongo
from pymongo import MongoClient

client = MongoClient('chatbotdb', 27017)


# Get the sampleDB database
# db = client.sampleDB

# Getting a Database

In [18]:
# db = client.test_database
db = client['test_database']


# Getting a Collection

In [19]:
# collection = db.test_collection
collection = db['test_collection']

# Documents

In [48]:
import datetime

################### auto increment ID
post = {'_id':collection.count_documents(filter={}),
        "author": "Mike",
         "text": "My first blog post!",
         "tags": ["mongodb", "python", "pymongo"],
         "date": datetime.datetime.utcnow()}

###################################################################
post = {
        "author": "Mike",
         "text": "My first blog post!",
         "tags": ["mongodb", "python", "pymongo"],
         "date": datetime.datetime.utcnow()}
post

{'_id': 2,
 'author': 'Mike',
 'text': 'My first blog post!',
 'tags': ['mongodb', 'python', 'pymongo'],
 'date': datetime.datetime(2018, 7, 20, 17, 57, 54, 556029)}

# Insert into Database

In [49]:
# posts = db.posts
post_id = collection.insert_one(post).inserted_id
post_id

2

# Find

In [32]:
a = []
for each in collection.find({'author':'Mike'},sort=[('date',-1)]):
    a.append(each)

In [27]:
a[0]

[{'_id': ObjectId('5b520f9f7f10e2033d2e6a68'),
  'author': 'Mike',
  'text': 'My first blog post!',
  'tags': ['mongodb', 'python', 'pymongo'],
  'date': datetime.datetime(2018, 7, 20, 16, 36, 43, 224000)},
 {'_id': ObjectId('5b5218607f10e2033d2e6a6a'),
  'author': 'Mike',
  'text': 'My first blog post!',
  'tags': ['mongodb', 'python', 'pymongo'],
  'date': datetime.datetime(2018, 7, 20, 17, 14, 7, 603000)},
 {'_id': ObjectId('5b5218867f10e2033d2e6a6b'),
  'author': 'Mike',
  'text': 'My first blog post!',
  'tags': ['mongodb', 'python', 'pymongo'],
  'date': datetime.datetime(2018, 7, 20, 17, 14, 30, 124000)}]

# count

In [38]:
collection.count_documents(filter={})

3

# DEV

In [1]:
import pymongo
from pymongo import MongoClient
import sys,os
sys.path.append('../Lib/')
sys.path.append('../ENV/')
from env import ENV
from LOG import Logger
import datetime as dt


class DB:
    def __init__(self,host=None,port=None,debug=False,id_increment=True,db=None,collection=None,enable=True):
        """
        host: mongo db host, default to chatbotdb
        port: mongo db port, default to 27017
        debug: False - in debug, data will be inserted into different Collections
        id_increment: if the id is increased numerally. default to True
        db: database default to 'chatbotdb'
        collection: default to chat
        """
        self.log = Logger(self.__class__.__name__,level=ENV.DB_LOG_LEVEL.value).logger
        self.debug = debug
        self.enable = enable
        self.id_increment=id_increment
        self._load_client(host,port)
        self._get_db(db)
        self._get_collection(collection)
        
    def _load_client(self,host=None,port=None):
        if host is None:
            host = 'chatbotdb'
        if port is None:
            port = 27017
        self.client = MongoClient(host,port)
        self.log.info('get mongodb client. host is:{}, port is: {}'.format(host,port))
            
    def _get_db(self,db=None):
        if db is None:
            db = 'chatbotdb'
        if self.debug:
            db = db + '_debug'
        self.db = self.client[db]
        self.log.info('mongodb database is: {}'.format(db))
        
            
    def _get_collection(self,collection=None):
        if collection is None:
            collection = 'chat'
        if self.debug:
            collection = collection + '_debug'
        self.collection = self.db[collection]
        self.log.info('mongodb collection is: {}'.format(collection))
        
    def drop_collection(self):
        textin = input('You are about to drop collection: {}. Please type in the name to confirm'.format(self.collection.name))
        if textin == self.collection.name:
            self.collection.drop()
            self.log.info('{} was dropped!'.format(self.collection.name))
            
    def drop_database(self):
        textin = input('You are about to drop database: {}. Please type in the name to confirm'.format(self.db.name))
        if textin == self.db.name:
            self.client.drop_database(self.db.name)
            self.log.info('DB {} was dropped!'.format(self.db.name))
            
    def insert(self,obj={}):
        if not self.enable:
            self.log.warning('database is not enabled!')
            return None
        obj.update({'lastUpdateDate':dt.datetime.utcnow()})
        if self.id_increment:
            obj.update({'_id':self.collection.count_documents(filter={})+1})
        insert_id = self.collection.insert_one(obj).inserted_id
        self.log.info('{} was inserted into collection: {}'.format(insert_id,self.collection.name))
        
    
        

In [2]:
d = DB(enable=False)

2018-07-23 04:11:55,034 - INFO - CLASS:DB- METHOD:_load_client -LINE:35 - MSG:get mongodb client. host is:chatbotdb, port is: 27017
2018-07-23 04:11:55,042 - INFO - CLASS:DB- METHOD:_get_db -LINE:43 - MSG:mongodb database is: chatbotdb
2018-07-23 04:11:55,046 - INFO - CLASS:DB- METHOD:_get_collection -LINE:52 - MSG:mongodb collection is: chat


In [3]:
d.insert({'user':'罗巍'})

2018-07-23 04:11:55,871 - WARNING - CLASS:DB- METHOD:insert -LINE:68 - MSG:database is not enabled!


In [8]:
d.drop_database()

You are about to drop database: chatbotdb. Please type in the name to confirmchatbotdb


2018-07-20 19:04:28,068 - INFO - CLASS:DB- METHOD:drop_database -LINE:63 - MSG:DB chatbotdb was dropped!


In [4]:
c = MongoClient('chatbot',27017)

In [6]:
d = c['ddd']

In [7]:
d.name

'ddd'

In [1]:
import sys,os
sys.path.append('../Lib/')
from MGODB import DB
from TIME import LocalDateTime

In [2]:
db = DB()

2018-07-22 20:08:06,547 - INFO - CLASS:DB- METHOD:_load_client -LINE:34 - MSG:get mongodb client. host is:chatbotdb, port is: 27017
2018-07-22 20:08:06,559 - INFO - CLASS:DB- METHOD:_get_db -LINE:42 - MSG:mongodb database is: chatbotdb
2018-07-22 20:08:06,578 - INFO - CLASS:DB- METHOD:_get_collection -LINE:51 - MSG:mongodb collection is: chat


In [3]:
t = LocalDateTime()
c = t.createLocalTime(year=2018,month=7,day=23,hour=15)
c

datetime.datetime(2018, 7, 23, 15, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>)

In [4]:
b = t.getLocalNow()
b

datetime.datetime(2018, 7, 23, 4, 8, 7, 697577, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>)

In [5]:
import pytz
import datetime as dt
tz = pytz.timezone("America/New_York")
ny_now = dt.datetime.utcnow()
ny_now = tz.localize(ny_now)
ny_now

datetime.datetime(2018, 7, 22, 20, 4, 8, 525139, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)

In [6]:
tz_sh = pytz.timezone("Asia/Shanghai")
sh_now = dt.datetime.utcnow()
sh_now = tz_sh.localize(sh_now)
sh_now

datetime.datetime(2018, 7, 22, 20, 4, 8, 863887, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>)

In [7]:
(sh_now - ny_now).total_seconds()/3600

-11.999905903333334

In [9]:
current = {'current_utc_ny':b,'current_utc_sh':sh_now}
db.insert(current)

2018-07-22 20:04:50,602 - INFO - CLASS:DB- METHOD:insert -LINE:70 - MSG:1 was inserted into collection: chat


In [43]:
finds = []
for each in db.collection.find({}):
    finds.append(each)

In [44]:
finds[-1]['current_utc_ny']

datetime.datetime(2018, 7, 22, 23, 53, 12, 663000)

In [4]:
for each in os.environ:
    print(each)

CUDNN_VERSION
HOSTNAME
NVIDIA_REQUIRE_CUDA
TERM
OLDPWD
LS_COLORS
NVIDIA_VISIBLE_DEVICES
LD_LIBRARY_PATH
NVIDIA_DRIVER_CAPABILITIES
NLTK_DATA
PATH
PWD
CUDA_PKG_VERSION
CUDA_VERSION
SHLVL
HOME
HostOS
LESSOPEN
LESSCLOSE
_
JPY_PARENT_PID
CLICOLOR
PAGER
GIT_PAGER
MPLBACKEND
